In [ ]:
import os
import time
import tqdm
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Patch, FancyArrowPatch
from matplotlib import colors
import logging
import yaml
import json
import joblib
from sklearn.impute import SimpleImputer
from scipy.stats.contingency import crosstab
import networkx as nx
from matplotlib.lines import Line2D
import matplotlib as mpl
from scipy.cluster.hierarchy import linkage, dendrogram
import seaborn as sns

sys.path.append(os.getcwd())
from functions.load_model import load_tolist
import functions.visualise as vis
import functions.process as proc
from functions.io import setup_logger, makedir


from scipy import stats
from statsmodels.sandbox.stats.multicomp import multipletests

In [ ]:
def load_data_from_keys(json_file, key):
    def traverse_dict(d, key):
        if isinstance(d, dict):
            if key in d.keys():
                return {key: d[key]}
            else:
                return {k: traverse_dict(v, key) for k, v in d.items()}
        elif isinstance(d, list):
            return [traverse_dict(x, key) for x in d]
        else:
            return d

    with open(json_file, 'r') as f:
        data = json.load(f)
    nested_dict = traverse_dict(data, key)
    return {(innerKey, outerKey): values for outerKey, innerDict in nested_dict.items() for innerKey, values in innerDict.items()} 

In [ ]:

import matplotlib.pyplot as plt
def sig_stars(p):
    if p <= .01:
        return min(int(-np.ceil(np.log10(abs(p)))),4) # should be correct needs checking
    elif p <= .05:
        return 1
    else:
        return 0

def find_common(*lists):
    common = [value for value in lists[0] if all(value in lst for lst in lists)]
    return common

def find_uncommon(*lists):
    all_items = [item for lst in lists for item in lst]
    common = find_common(*lists)
    uncommon = [item for item in all_items if item not in common]
    return uncommon

def find_uncommon(*lists):
    all_items = np.array(lists)
    df_items = pd.DataFrame(lists)
    common = find_common(*all_items)
    common_bool = np.isin(df_items,common)
    uncommon = []
    for i,col in enumerate(common_bool):
        col_uncommon = df_items.loc[i,~col]
        uncommon.append('_'.join(col_uncommon.dropna()))
    return uncommon

Please provide where your files are stored and where you would like your data to be saved in the following section.

In [ ]:
datestr = time.strftime("%Y%m%d-%Hh%Mm")
inpath = "/gpfs/soma_fs/home/boeger/PpaPred/ErenBoeger_2024/" 
#inpath = '/gpfs/soma_fs/home/boeger/PpaPred/PpaPred_roca_35727184'

config_path = "config.yml"

# Eren Boeger 2024
include, fig_folder = ["Exp1_WT_larvae", "Exp1_WT_OP50"], "Exp1"   
#include, fig_folder = ["Exp2_WT_larvae", "Exp2_tph1_larvae", "Exp2_cat2_larvae", "Exp2_tdc1_larvae", "Exp2_tbh1_larvae"], "Exp2"
include, fig_folder = ["Exp3_tbh1_larvae", "Exp3_tdc1_larvae", "Exp3_tyra2_larvae", "Exp3_ser2_larvae", "Exp3_tyra3_larvae", "Exp3_lgc55_larvae"], "Exp3b"
include, fig_folder = ["Exp3_WT_larvae", "Exp3_tbh1_larvae", "Exp3_ser3_larvae", "Exp3_ser6_larvae", "Exp3_octr1_larvae"], "Exp3a"
include, fig_folder = ["Supp5_WT_larvae", "Supp5_tyramine_larvae", "Supp5_octopamine_larvae"], "Supp5"
include, fig_folder = ["Exp2_WT_larvae", "Supp4_tbh1tdc1_larvae"], "Supp4"
include, fig_folder = ["Exp3_WT_larvae", "Supp7_ser2tyra2tyra3_larvae"], "Supp7"

#include, fig_folder = ['L147', 'L157', 'L176', 'L118', 'L156'], 'fig_folder' #['L118','L147','L156','L157','L176']
#include, fig_folder  = ["L147", "L157", "L176", "L118", "L119", "L156"], 'Fig'

outpath = makedir(os.path.join(inpath,fig_folder))
file_pattern = 'batch.json'
stat_pop = f'{include[0]}_batch.json'
inpath_with_subfolders = True
use_bonferroni = True

WT_ordering = False #[1., 0., 2., 6., 8., 3., 4., 7., 5.]
plot = False

In [ ]:
#all_transitions = None
#all_durations = None

### I/O ################################################
all_files = [os.path.join(root, name) for root, dirs, files in os.walk(inpath) for name in files if file_pattern in name and any([c in name for c in include])]
all_files.sort(key = lambda i: np.where([c in i for c in include])) # to sort all_files as given in include
loc_all = {os.path.basename(f):f for f in all_files}

if outpath is None:
    outpath = os.path.commonpath(all_files)
    
### Configuration ################################################
config = yaml.safe_load(open(config_path, "r"))
cluster_color = config['cluster_color']
cluster_group = config['cluster_group_man']
cluster_label = config['cluster_names']
clu_group_label = {_:f'{_}, {__}' for _, __ in tuple(zip([c for c in cluster_label.values()],[g for g in cluster_group.values()]))}
skip_already = config['settings']['skip_already']
fps = 30
pctl_toplot = (0,100)
showfliers=False
loc_all

In [ ]:
mpl.rcParams['pdf.fonttype'] = 42
test_metrics = ['mean duration', 'rel time in']#'mean transitions' #'mean duration',
for metric in test_metrics:
    outmetric = os.path.join(outpath, f"{datestr}_{''.join(metric.split(' '))}_Utest")
    
    metric_multi = None
    for fn, fpath in loc_all.items():
        batch_metric = pd.DataFrame(load_data_from_keys(fpath, metric)).droplevel(0, axis=1)
        batch_metric.columns = pd.MultiIndex.from_product([[fn], batch_metric.columns])
        if metric_multi is None:
            metric_multi = batch_metric
        else:
            metric_multi = pd.concat([metric_multi, batch_metric], axis=1)
        #batch_etho = load_data_from_keys(fpath, 'ethogram')
    if metric == 'mean duration':
        metric_multi = metric_multi/fps
    metric_multi.index.name = 'cluster'
    if '-1' or -1 in metric_multi.index:
        try:
            metric_multi = metric_multi.drop(-1)
        except:
            try:
                metric_multi = metric_multi.drop('-1')
            except:
                pass
    if metric == 'rel time in':
        metric_multi = metric_multi/metric_multi.sum(axis=0)
        #print(metric_multi.groupby(level=0, axis=1).mean())
    #metric_multi.index = [eval(i) for i in metric_multi.index]
        
    ### stats
    
    all_ps_adju = pd.DataFrame(index=metric_multi.index)
    all_ps_orig = pd.DataFrame(index=metric_multi.index)
    all_conds = metric_multi.columns.get_level_values(0).unique()
    N_tests = len(all_conds)-1#len(test_metrics) if not 'mean transitions' in test_metrics else len(test_metrics)+len(cluster_label)-1
    print(N_tests)

    stats_csv = pd.DataFrame([], columns = ['Experiment', 'Population', 'Condition', 'State', 'N', 'Mean Rank', 'Sum Rank', 'U1', 'U2', 'Mann-Whitney U', 'p', 'number of tests', 'bonferroni p'])
    for i, cond in enumerate(all_conds):
        U1s, ps = stats.mannwhitneyu(metric_multi[stat_pop], metric_multi[cond], axis=1, nan_policy='omit')
        
        ps_adjusted = ps*N_tests # bonferroni correction... like this correction done for each cluster and each condition seperatly across metrics tested
        all_ps_adju[cond] = ps_adjusted
        all_ps_orig[cond] = ps

        # get number of conditions
        cond_n = np.count_nonzero(~np.isnan(metric_multi[cond]), axis=1)
        pop_n = np.count_nonzero(~np.isnan(metric_multi[stat_pop]), axis=1)
        # calculuate U2 from U1
        U2s = pop_n*cond_n - U1s
        Us = np.min((U1s, U2s), axis=0)

        # rank dataset, U1 or U2 belong to?, but strictly important, because U = min(U1,U2)
        rank_multi = pd.concat([metric_multi[stat_pop], metric_multi[cond]], axis=1,
                               keys={'pop':metric_multi[stat_pop].columns, 'cond':metric_multi[cond].columns,})
        cond_idx, = np.where(rank_multi.columns.get_level_values(0)=='cond')
        ranks = stats.rankdata(rank_multi, axis=1, nan_policy='omit')
        cond_ranks = ranks[:,cond_idx]
        cond_ranks_mean = np.nanmean(cond_ranks, axis=1)
        cond_ranks_sum = np.nansum(cond_ranks, axis=1)

        # create dict of current condition
        stats_cond = {'Experiment': [fig_folder]*metric_multi.shape[0], 
                      'Population': [stat_pop]*metric_multi.shape[0], 
                      'Condition': [cond]*metric_multi.shape[0], 
                      'State': [cluster_group[eval(c)] for c in metric_multi.index], 
                      'N': cond_n, 
                      'Mean Rank': cond_ranks_mean, 
                      'Sum Rank': cond_ranks_sum, 
                      'U1': U1s, 
                      'U2': U2s, 
                      'Mann-Whitney U': Us, 
                      'p': ps, 
                      'number of tests': [N_tests]*metric_multi.shape[0], 
                      'bonferroni p': ps_adjusted}
        # concat stats dict to dataframe stats_csv
        stats_csv = pd.concat([stats_csv, pd.DataFrame(stats_cond)])

    stats_csv.to_csv(outmetric+'.csv')

    if use_bonferroni:
        all_ps = all_ps_adju
    else:
        all_ps = all_ps_orig
    ### make boxplots
    if metric != 'mean transitions':
        percentile = (np.round(np.nanpercentile(metric_multi,pctl_toplot[0])).astype(int),np.round(np.nanpercentile(metric_multi,pctl_toplot[1])).astype(int))

        fig, ax = plt.subplots(figsize= (6*len(all_ps.columns),4))
        N_across = []
        for i, cond in enumerate(all_conds):
            N_across.append(np.count_nonzero(~np.isnan(metric_multi[cond]), axis=1))
            for j,c in enumerate(metric_multi.index):
                c_ = eval(c)
                c_ = c_[0] if isinstance(c_, tuple) else c_
                p = all_ps.loc[c, cond]
                hpos = j*len(all_conds)+i*.8
                ax.boxplot(metric_multi[cond].loc[c][~np.isnan(metric_multi[cond].loc[c])], 
                            positions = [hpos],
                            widths=.4,
                            showfliers=showfliers,
                            #showmeans =True, meanline = True,
                            patch_artist = True, boxprops={'facecolor':cluster_color[c_]},medianprops={'color':'k'})
        
        for i, cond in enumerate(all_conds):
            for j,c in enumerate(metric_multi.index):
                p = all_ps.loc[c, cond]
                hpos = j*len(all_conds)+i*.8
                vpos = ax.get_ylim()[1]
                if p < .05:
                    #percentile[1]
                    #vpos = vpos+vpos*.05*i #np.max(metric_multi.loc[str(c), cond])+
                    ax.text(hpos, vpos,'*'*sig_stars(p), ha='left', va='bottom', rotation=45)
                else:
                    ax.text(hpos, vpos,'n.s.', ha='left', va='bottom', rotation=45)
                ax.text(hpos, vpos*1.15,'N='+str(N_across[i][j]),ha='center', va='bottom')
                    
        #for c in metric_multi.index:
        #    c_ = eval(c)
        #    c_ = c_[0] if isinstance(c_, tuple) else c_
        #    ax.text(c_*len(all_conds), -ax.get_ylim()[1]*.5, cluster_group[c_], ha='left', va='top', rotation=-45)
                #plt.text(c+i*.2, texty, np.round(p,3))

        common = (find_common(*[s.split('_') for s in include]))
        uncommon = (find_uncommon(*[s.split('_') for s in include]))
        #uncommon = [u for u in uncommon if not 'Exp' in u]

        xlabels = np.repeat(uncommon,len(cluster_color)-1)
        ax.set_xticklabels(xlabels, rotation=90)        
        ax.set_ylabel(f'{metric}')
        #prange5p = np.diff(percentile)*.05
        #ax.set_ylim(percentile[0]-prange5p, percentile[1]+prange5p)
        ax.spines[['right', 'top']].set_visible(False)
        txt=json.dumps(stats_json['meta'],separators=('\n',','))#+'\nshown range in 0.5 to 99.5 percentile'
        plt.title('_'.join(common),x=1.1,y=1.1)
        plt.figtext(1, .9, txt, wrap=True, ha='left', va='center', rotation=-90)
        #plt.savefig(outmetric+'.pdf', bbox_inches="tight")
        plt.show()
    else:
        metric_multi.index = pd.MultiIndex.from_tuples([eval(i) for i in metric_multi.index])
        metric_multi = metric_multi.drop(-1, level = 0)
        metric_multi = metric_multi.drop(-1, level = 1)
        all_ps.index = pd.MultiIndex.from_tuples([eval(i) for i in all_ps.index])
        ind_lvl = metric_multi.index.get_level_values(0).unique()
        fig, axs = plt.subplots(len(ind_lvl), figsize= (13,12), sharex=True)
        for i, c_from in enumerate(ind_lvl):
            for j,cond in enumerate(all_conds):
                for k, c_to in enumerate(ind_lvl):
                    if i == k:
                        continue
                    p = all_ps[cond].xs((c_from, c_to))
                    hpos = k*len(all_conds)+j*.8
                    axs[i].boxplot(metric_multi[cond].xs((c_from, c_to))[~np.isnan(metric_multi[cond].xs((c_from, c_to)))], 
                                positions = [hpos],
                                widths=.4,
                                showfliers=showfliers,
                                patch_artist = True, boxprops={'facecolor':cluster_color[c_to]},medianprops={'color':'k'})
                    if p <= .05:
                        axs[i].text(hpos, .9,'*'*sig_stars(p), ha='left', va='bottom', rotation=45)
                    else:
                        axs[i].text(hpos, .9,'n.s.', ha='left', va='bottom', rotation=45)
            axs[i].set_ylim(0,1)
            axs[i].set_ylabel(f'{cluster_group[c_from]}')
            axs[i].spines[['right', 'top']].set_visible(False)
            #axs[-1].text(i*len(all_conds)+len(all_conds)/3, -.9, cluster_group[c_from], ha='center', va='center', rotation=-0)
            
        #axs[0].set_title(metric)
        axs[-1].set_xticklabels(np.tile(np.repeat([s.split('_')[1] for s in include],len(cluster_color)-2),len(cluster_color)-1), rotation=-90)#-2 in first repeat because without none and self
        #plt.savefig(outmetric+'.pdf', bbox_inches="tight")
        plt.show()

        fig, ax = plt.subplots(figsize=(6*len(all_ps.columns),4))

        for i, c_ in enumerate(ind_lvl):
            for j,cond in enumerate(all_conds):
                p = all_ps[cond].xs((c_, c_))
                hpos = i*len(all_conds)+j*.8
                d = metric_multi[cond].xs((c_, c_))[~np.isnan(metric_multi[cond].xs((c_, c_)))]
                ax.boxplot(d, 
                            positions = [hpos],
                            widths=.4,
                            showfliers=showfliers,
                            patch_artist = True, boxprops={'facecolor':cluster_color[c_]},medianprops={'color':'k'})
                if p < .05:
                    ax.text(hpos, 1,'*'*sig_stars(p), ha='left', va='bottom', rotation= 45)
                else:
                    ax.text(hpos, 1,'n.s.', ha='left', va='bottom', rotation=45)
                ax.text(hpos, 1.15,'N='+str(len(d)),ha='center', va='bottom')
                        
            ax.set_ylim(0,1)
            ax.set_ylabel(f'mean persistance (self-loop)')
            ax.spines[['right', 'top']].set_visible(False)
            #ax.text(i*len(all_conds)+len(all_conds)/3, -ax.get_ylim()[1]*.5, cluster_group[c_], ha='center', va='top', rotation=-45)
        
        ax.set_xticklabels(np.tile([s.split('_')[1] for s in include],len(cluster_color)-1), rotation=-0)
        #plt.savefig(outmetric+'_self.pdf', bbox_inches="tight")
        plt.show()
    

In [ ]:
def common_substring(lst, splitter='_'):
    compare_to = lst[0].split(splitter)
    common, uncommon = [], []
    for string in lst[1:]:
        subs = string.split(splitter)
        for s in subs:
            if s in compare_to:
                common.append(s)
            else:
                uncommon.append(s)
    uncommon += compare_to.remove(common)
    return common, uncommon

# Test the function
print(common_substring(include))

In [ ]:
lst1 = ['this', 'that', 'these']
lst2 = ['this', 'that', 'those']
lst3 = ['this', 'that', 'them']
common = (find_common(*[s.split('_') for s in include]))  # Output: [5]
uncommon = (find_uncommon(*[s.split('_') for s in include]))

In [ ]:
for metric in ['mean transitions']:
    outmetric = os.path.join(outpath, f"{datestr}_{''.join(metric.split(' '))}_Utest")
    
    metric_multi = None
    for fn, fpath in loc_all.items():
        batch_metric = pd.DataFrame(load_data_from_keys(fpath, metric)).droplevel(0, axis=1)
        batch_metric.columns = pd.MultiIndex.from_product([[fn], batch_metric.columns])
        if metric_multi is None:
            metric_multi = batch_metric
        else:
            metric_multi = pd.concat([metric_multi, batch_metric], axis=1)
        #batch_etho = load_data_from_keys(fpath, 'ethogram')
    if metric == 'mean duration':
        metric_multi = metric_multi/fps
    metric_multi.index.name = 'cluster'

    all_conds = metric_multi.columns.get_level_values(0).unique()
    id_cond = [c.split('_')[0] for c in all_conds]
    metric_multi.index = pd.MultiIndex.from_tuples([eval(i) for i in metric_multi.index])
    metric_multi = metric_multi.drop(-1, level = 0)
    metric_multi = metric_multi.drop(-1, level = 1)
    for i, cond in enumerate(all_conds):
        pass
    break

In [ ]:
fr_transition = metric_multi['bac_data_batch.json'].mean(axis=1).unstack(level=0)
fr_transition

In [ ]:
tr_larvae = metric_multi['larvae_data_batch.json'].mean(axis=1).unstack(level=0)
tr_larvae_norm = (tr_larvae/tr_larvae.sum(axis=0)).fillna(0)
plt.imshow(tr_larvae_norm)

In [ ]:
testbatch = 'tbh1_larvae_batch.json'
fr_transition = metric_multi[testbatch].mean(axis=1).unstack(level=0)
fr_transition_norm = (fr_transition/fr_transition.sum(axis=0)).fillna(0)
plt.imshow(fr_transition_norm)

In [ ]:
trans_rel = fr_transition_norm-tr_larvae_norm
plt.imshow(trans_rel, cmap='seismic',vmin=-.5,vmax=.5)
plt.title(f'{testbatch} against larvae')

In [ ]:
stats_json = {'meta':{'metric': metric,
                          'statistical test': 'Mann-Whitney-U-Test, two-sided',
                          'bonferroni correction': 'statsmodels...multipletests',
                          'population': stat_pop},
              'adjusted ps': all_ps.to_dict(),
              'Mann-Whitney U': all_U1s.to_dict(),
              'unadjusted ps': all_ps_orig.to_dict(),
              }
jsnF = json.dumps(stats_json, indent = 4)
with open(os.path.join(outpath, f'{datestr}_{metric}_Utest.json'), "w") as outfile:
    outfile.write(jsnF)

In [ ]:
li = [eval(i) for i in metric_multi.index]
type(li[0])

In [ ]:
op.itemgetter('metric', 'statistical test', 'bonferroni correction', 'population')(stats_json)
op.getitem(stats_json,'metric')

In [ ]:
from scipy import stats
U1s, pvs = stats.mannwhitneyu(metric_multi[stat_pop], metric_multi['bac_data_batch.json'], axis=1, nan_policy='omit')

In [ ]:

___,ps_adjusted,__,_ = multipletests(pvs, method='bonferroni')
np.round(ps_adjusted,5)

In [ ]:


    ### Load data ################################################
    #data_batch = load_tolist(fpath, droplabelcol=False)
    #data_batch_concat = pd.concat([d for d in data_batch], axis=0)
    
    #y_batch_concat = data_batch_concat['prediction']
    #y_batch = [d['prediction'] for d in data_batch]

    
    ### Batch summary ############################################
    for i,d in enumerate(loc_summ):
        summ_ = pd.read_csv(d)
        dev_ = (~summ_.isna()).astype(int)
        summ_ = summ_.fillna(0)
        #trans_col_ = [[c for c in fr_transition_],[c for c in fr_transition_]]
        
        if i == 0:
            summ = summ_
            devider = dev_
        else:
            summ += summ_
            devider += dev_
    summary = summ/devider
    #summary.to_csv(os.path.join(outpath, f'{data_str}_batch_summary.csv'))
    
    total_dur = summary.duration_mean*summary.duration_count
    total_dur_rel = total_dur/total_dur.sum()
    
    ### Batch transitions ############################################
    #batch_transitions = np.full((len(loc_trans),9,9), np.nan) # with shape recordings,cluster from, cluster to        
    for i,d in enumerate(loc_trans):
        fr_transition_ = pd.read_csv(d, index_col=0)
        #batch_transitions[i] = fr_transition_
        trans_col_ = [[c for c in fr_transition_],[c for c in fr_transition_]]
        if i == 0:
            fr_transition = fr_transition_
            trans_col = trans_col_
        elif trans_col_ == trans_col:
            #print('this')
            fr_transition += fr_transition_
        else:
            print('WARNING')

    # merge all transitions across animals and experiments
    #if all_transitions is None:
    #    all_transitions = np.array([batch_transitions])
    #else:
    #    all_transitions,batch_transitions = padtoequalsize(all_transitions, batch_transitions, [-3])
    #     all_transitions = np.concatenate([all_transitions,[batch_transitions]],axis=0)

    # calculate the normalised transition and cluster
    # cluster behaviors with single linkage
    Z = linkage(fr_transition, 'single', optimal_ordering=True)
    # extract closest cluster
    ordering = np.concatenate((Z[::-1,0],Z[:,1]))
    ordering = ordering[ordering<len(cluster_label)]-1
    # calculate normalised transitions
    fr_trans_norm = (fr_transition/fr_transition.sum(axis=0)).fillna(0)
    fr_transition_clust = fr_trans_norm.iloc[ordering,ordering]
    fr_trans_norm.to_csv(os.path.join(outpath, f'{data_str}_batch_transitions_2.csv'), index=False)
    



    ### Batch durations ############################################
    #batch_durations = None
    for rec in loc_onoff:
        with open(rec,'r') as jf:
            onoff = json.load(jf)
        #duration = pd.DataFrame(dtype='float64')
        for cluster in onoff: #for bout in cluster
            cluster_dur = []
            for bout in onoff[cluster]: #for bout in cluster
                cluster_dur.append(bout[1])
            #duration = pd.concat([duration,pd.DataFrame([cluster_dur],index=[eval(cluster)],dtype='float64')],axis=0)
            
        cl_present = duration.index
        cl_range = range(min(cl_present), max(cl_present)+1)
        duration = duration.reindex(cl_range)
        
        if batch_durations is None:
            batch_durations = np.array([duration.values])
        else:
            batch_durations, duration = padtoequalsize(batch_durations, duration, [-1,-2])
            batch_durations = np.concatenate([batch_durations,[duration]],axis=0)
    
    if all_durations is None:
        all_durations = np.array([batch_durations])
    else:
        all_durations,batch_durations = padtoequalsize(all_durations, batch_durations, [-1,-2,-3])
        all_durations = np.concatenate([all_durations,[batch_durations]],axis=0)
    
        
    #############
    ### plots ###
    #############
    
    if plot:
        """
        transition_plot = vis.transition_plotter(fr_trans_norm.values, cluster_color, node_alpha=summary['duration_relative'].fillna(0).tolist())
        plt.text(1.5, -1, f'{data_str}\nN = {len(loc_trans)}', fontsize=12)
        plt.title(f"transitions of {data_str}")
        plt.savefig(os.path.join(outpath, f'{data_str}_batch_transitions.pdf'))
        plt.show()"""
        
        fig = plt.figure()
        axs1 = fig.add_axes([0, .895, .2, .805])
        axs1.axis('off')
        dn = dendrogram(Z, orientation='left',ax= axs1)
        axs2 = fig.add_axes([0.33, .9, .8, .8])
        im = axs2.imshow(fr_transition_clust,norm=colors.PowerNorm(.4,vmax=1))#vmin=0,vmax=.2)#
        axs2.set_xticks(range(len(cluster_group)))
        axs2.set_xticklabels([cluster_group[k] for k in ordering], rotation=45,ha="center")
        axs2.set_yticks(range(len(cluster_group)))
        axs2.set_yticklabels([cluster_group[k] for k in ordering])
        cbar = axs2.figure.colorbar(im, ax=axs2)
        cbar.ax.set_ylabel("X^0.4 normalization", rotation=90, labelpad= 6)
        axs2.set_title(f"transitions of {data_str} per sec (x to y)")
        plt.savefig(os.path.join(outpath, f'{data_str}_batch_transitheatmap_clust.pdf'),bbox_inches = "tight")
        plt.show()
    
        if WT_ordering:
            fr_transition_WT = fr_trans_norm.iloc[WT_ordering,WT_ordering]
            fig = plt.figure()
            axs1 = fig.add_axes([0, .895, .2, .805])
            axs1.axis('off')
            #dn = dendrogram(Z, orientation='left',ax= axs1)
            axs2 = fig.add_axes([0.33, .9, .8, .8])
            im = axs2.imshow(fr_transition_WT,norm=colors.PowerNorm(.4,vmax=1))#vmin=0,vmax=.2)#
            axs2.set_xticks(range(len(cluster_group)))
            axs2.set_xticklabels([cluster_group[k] for k in WT_ordering], rotation=45,ha="center")
            axs2.set_yticks(range(len(cluster_group)))
            axs2.set_yticklabels([cluster_group[k] for k in WT_ordering])
            cbar = axs2.figure.colorbar(im, ax=axs2)
            cbar.ax.set_ylabel("X^0.4 normalization", rotation=90, labelpad= 6)
            axs2.set_title(f"transitions of {data_str} per sec (x to y)")
            plt.savefig(os.path.join(outpath, f'{data_str}_batch_transitheatmap_WTclust.pdf'),bbox_inches = "tight")
            plt.show()
        
    
        
        fig, ax = plt.subplots(figsize=(2,5))
        b = ax.barh(range(len(total_dur_rel)),total_dur_rel, color=list(cluster_color.values())[1:])
        ax.set_yticks(range(len(cluster_group)-1))
        ax.set_yticklabels([cluster_group[k] for k in cluster_group][1:])
        ax.bar_label(b, label_type='edge', fmt='%.2g', padding=3)
        ax.invert_yaxis()
        plt.xlabel(f"total rel. duration")
        plt.title(data_str)
        plt.xlim(0,.6)
        plt.savefig(os.path.join(outpath, f'{data_str}_batch_totaldur.pdf'),bbox_inches = "tight")
        plt.show()
    
        # Calculate onoff sets
        onoff = proc.onoff_dict(y_batch, labels =np.unique(y_batch_concat))
    
        fig = plt.figure(figsize=(.6*len(onoff.keys()),3), layout='constrained')
        for k in onoff.keys():
            if not np.isnan(k):
                durs = pd.DataFrame(onoff[k])[1]/30
                plt.boxplot(durs, positions=[k], widths=.5,sym='', patch_artist = True, boxprops={'facecolor':cluster_color[k]},medianprops={'color':'k'})
        plt.xticks(range(len(cluster_group)-1),[cluster_group[k] for k in cluster_group][1:], rotation=45)
        plt.ylabel("sec")
        plt.ylim(0,30)
        plt.title(f"duration of {data_str} (without fliers)")
        plt.savefig(os.path.join(outpath, f'{data_str}_batch_durboxplot.pdf'),bbox_inches = "tight")
        plt.show()

In [ ]:
all_durations.shape

In [ ]:
cl_present

In [ ]:
def df_withnans_alignright(df_):
    df = df_.copy()
    for i in range(len(df.columns)):
        nulls = df.iloc[np.where(df.iloc[:,i].notnull())[0][-1]+1:,i]
        notnulls = df.iloc[:np.where(df.iloc[:,i].notnull())[0][-1],i]
        right_aligned = pd.concat([nulls,notnulls], axis=0).reset_index(drop=True)
        df.iloc[:,i] = right_aligned
    return df

In [ ]:
np.random.seed(96)
bout_all = pd.DataFrame([])
bout_pre_all = pd.DataFrame([])
y_pre_all = pd.DataFrame([])
bout_post_all = pd.DataFrame([])
y_post_all = pd.DataFrame([])
feat = 'velocity'
cl = 1
rdm_subset = np.random.choice(len(onoff[cl]), 100, replace=False)
rdm_ons =[]
for i,oo in enumerate(rdm_subset):
    rdm_ons.append(onoff[cl][oo][0])
    onset = onoff[cl][oo][0]
    offset = onoff[cl][oo][0]+onoff[cl][oo][1]
    bout = data_batch_concat.reset_index(drop=True)[onset:offset].reset_index(drop=True)
    if all(bout[feat].isnull()):
        print(i)
        continue
    bout_pre = data_batch_concat.reset_index(drop=True)[onset-60:onset].reset_index(drop=True)
    bout_post = data_batch_concat.reset_index(drop=True)[offset:offset+60].reset_index(drop=True)
    
    bout_all = pd.concat([bout_all,bout[feat]], axis=1)

    bout_pre_all = pd.concat([bout_pre_all,bout_pre[feat]], axis=1)
    y_pre_all = pd.concat([y_pre_all,bout_pre['prediction']], axis=1)
    bout_post_all = pd.concat([bout_post_all,bout_post[feat]], axis=1)
    y_post_all = pd.concat([y_post_all,bout_post['prediction']], axis=1)

bout_all.columns = list(range(len(bout_all.columns)))
bout_pre_all.columns = list(range(len(bout_pre_all.columns)))
y_pre_all.columns = list(range(len(y_pre_all.columns)))
y_pre_color = y_pre_all[:150].replace(cluster_color)
bout_post_all.columns = list(range(len(bout_post_all.columns)))
y_post_all.columns = list(range(len(y_post_all.columns)))
y_post_color = y_post_all[:150].replace(cluster_color)

bout_all_right = df_withnans_alignright(bout_all)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
mask = data_batch_concat.prediction.isin([7])
#fig = px.lines(bout_all, color = y_bout_all, width=800, height=400)
fig = make_subplots(rows=1, cols=2)
for trace in bout_all:
    bout_ = bout_all[trace].iloc[:90].dropna().reset_index(drop=True)
    bout_r_ = bout_all_right[trace].iloc[-90:].dropna().reset_index(drop=True)
    pre_ = bout_pre_all[trace][y_pre_all[trace]==y_pre_all[trace].iloc[-1]].reset_index(drop=True)
    post_ = bout_post_all[trace][y_post_all[trace]==y_post_all[trace].iloc[-1]].reset_index(drop=True)
    
    fig.add_trace(go.Scatter(x=np.arange(len(bout_)), y=bout_, line=dict(color=cluster_color[cl]),mode='lines'),
                 row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(-len(pre_),0), y=pre_, line=dict(color=y_pre_color[trace].iloc[-1]),mode='lines'),
                 row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(-len(bout_r_),0), y=bout_r_, line=dict(color=cluster_color[cl]),mode='lines'),
                 row=1, col=2)
    fig.add_trace(go.Scatter(x=np.arange(len(post_)), y=post_, line=dict(color=y_post_color[trace].iloc[-1]),mode='lines'),
                 row=1, col=2)

    
mean_ = pd.concat([bout_pre_all,bout_all],axis=0).reset_index(drop=True).mean(axis=1).iloc[:150]
mean_r_ = pd.concat([bout_all_right, bout_post_all],axis=0).reset_index(drop=True).mean(axis=1).iloc[-150:]
fig.add_trace(go.Scatter(x=np.arange(-60,90), y=mean_, line=dict(color='black',width=3),mode='lines'),
                 row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(-90,60), y=mean_r_, line=dict(color='black',width=3),mode='lines'),
                 row=1, col=2)

fig.add_vline(x=0, line=dict(color="Grey",width=5))
fig.update_xaxes(dtick=15, title=dict(text='Time [frames]'))
fig.update_yaxes(range=[0, max(bout_all.max().max(),bout_pre_all.max().max(),bout_post_all.max().max())],title=dict(text=f'{feat}'))
fig.update_layout(
    width=1200,
    height=500,
    xaxis = dict(
        tickmode = 'linear',
        dtick = 15),
    showlegend=False,
    template='plotly_white',
    title=dict(text=f"Traces of {cluster_group[cl]} aligned at prediction onset (l) and offset (r)", font=dict(size=16), x =.5, xanchor='center'))
fig.show()

In [ ]:
pd.concat([bout_pre_all,bout_all], axis=0).reset_index(drop=True)[:150]

In [ ]:
onoff_ = pd.json_normalize(onoff).to_dict(orient='split')
onoff_rev = {}
for l,c in zip(onoff_['data'][0],onoff_['columns']):
    for oo in l:
        onoff_rev[oo[0]] = (c, oo[1])

onoff_rev_df = pd.DataFrame(onoff_rev).T
onoff_rev_df = onoff_rev_df.reindex(sorted(onoff_rev_df.index), axis=0)

onoff_rev_dfidx = onoff_rev_df.index.to_series().reset_index(drop=True)
prebout_idx = (onoff_rev_dfidx[onoff_rev_dfidx.isin(rdm_ons)].index - 1).tolist()

prebout_c, prebout_dur, prebout = onoff_rev_df.iloc[prebout_idx,0],onoff_rev_df.iloc[prebout_idx,1], onoff_rev_df.index[prebout_idx].to_list()

In [ ]:
fig, axs = plt.subplots(1,2, figsize=(8,5), sharey=True)
axs[0].plot(range(0,90),bout_all[:90], c=cluster_color[cl],alpha=.7)
for b,d,c in zip(prebout,prebout_dur,prebout_c):
    axs[0].plot(range(-d if d < 60 else -60,0),data_batch_concat.reset_index(drop=True).iloc[b:b+d][feat][-d if d < 60 else -60:], c=cluster_color[c],alpha=.7)
axs[0].plot(range(0,90),bout_all[:90].median(axis=1), c='#252422', linewidth=2)
axs[0].axvline(0)
axs[0].set_title("aligned beginning")
axs[1].plot(range(-90,0),bout_all_right[-90:], c=cluster_color[cl],alpha=.7)
axs[1].plot(range(-90,0),bout_all_right[-90:].median(axis=1), c='#252422', linewidth=2)
axs[1].axvline(0)
axs[1].set_title("aligned end")

In [ ]:
y_pre_all[trace][y_pre_all[trace]==y_pre_all[trace].iloc[-1]]

In [ ]:
import scipy
from scipy.cluster.hierarchy import linkage, dendrogram
Z = linkage(fr_transition, 'single', optimal_ordering=True)
ordering = np.concatenate((Z[::-1,0],Z[:,1]))
ordering = ordering[ordering<=8]
fr_trans_norm = (fr_transition/fr_transition.sum(axis=0)).fillna(0)
fr_transition_clust = fr_transition.iloc[ordering,ordering]
fr_trans_clust_norm = 

In [ ]:
fig = plt.figure()
axs1 = fig.add_axes([0, .895, .2, .805])
axs1.axis('off')
dn = dendrogram(Z, orientation='left',ax= axs1)
axs2 = fig.add_axes([0.33, .9, .8, .8])
im = axs2.imshow(fr_transition_clust,norm=colors.PowerNorm(.4,vmax=1))#vmin=0,vmax=.2)#
axs2.set_xticks(range(len(cluster_group)-1))
axs2.set_xticklabels([cluster_group[k] for k in ordering], rotation=45,ha="center")
axs2.set_yticks(range(len(cluster_group)-1))
axs2.set_yticklabels([cluster_group[k] for k in ordering])
cbar = axs2.figure.colorbar(im, ax=axs2)
cbar.ax.set_ylabel("X^0.4 normalization", rotation=90, labelpad= 6)
axs2.set_title(f"transitions of {data_str} per sec (x to y)")
plt.savefig(os.path.join(out_predicted, os.path.basename(outpath)+'_batch_transitheatmap_clust.pdf'),bbox_inches = "tight")
plt.show()

In [ ]:
fr_trans_norm

In [ ]:
fig, ax = plt.subplots(figsize=(2/.6*1,5))
b = ax.barh(range(len(total_dur_rel)),total_dur_rel, color=list(cluster_color.values())[1:])
ax.set_yticks(range(len(cluster_group)-1))
ax.set_yticklabels([cluster_group[k] for k in cluster_group][1:])
ax.bar_label(b, label_type='edge', fmt='%.2g', padding=3)
ax.invert_yaxis()
plt.xlabel(f"total rel. duration")
plt.title(data_str)
plt.xlim(0,1)
plt.savefig(os.path.join(out_predicted, os.path.basename(outpath)+'_batch_totaldur.pdf'),bbox_inches = "tight")
plt.show()

In [ ]:
    data_describe = data_batch_concat.groupby(y_batch_concat).describe().T.loc[idx[:, ['mean','std','count']], :].sort_index(level=0).T
    dur_describe = pd.DataFrame(dur, columns=['duration']).groupby(transi).describe().T.loc[idx[:, ['mean','std','count']], :].sort_index(level=0).T
    dur_describe['duration','relative'] = pd.DataFrame(dur, columns=['duration']).groupby(transi).apply(lambda cd: cd.sum()/len(y_batch_concat))
    summary = pd.concat([dur_describe, data_describe], axis=1)
    summary.index.name = 'cluster'
    summary = summary.T.reset_index(drop=True).set_index(summary.T.index.map('_'.join)).T
    summary = summary.set_index(summary.index.astype(int))
    summary = summary.reindex([k for k in cluster_label if k != -1])
    summary.to_csv(os.path.join(out_predicted, os.path.basename(outpath)+'_batch_summary.csv'))

    #### Older Version
    
    for i,d in enumerate(data_batch):
        frame = d['prediction'].rolling(30).apply(lambda s: s.mode()[0])[29::30].values.flatten()
        trans_col_,fr_transition_ = crosstab(frame[1:], frame[:-1], levels=([k for k in cluster_label if k != -1],[k for k in cluster_label if k != -1]))
        #fr_transition_ = pd.read_csv() ##################################read transitions.csv should look like normal fr_transition, get trans_col from header
        if i == 0:
            fr_transition = fr_transition_
            trans_col = trans_col_
        if trans_col_ == trans_col:
            fr_transition += fr_transition_
        else:
            print('WARNING')
        #fr_transition/fr_transition.sum(axis=0)
    
    #othersum_axis0 = fr_transition.sum(axis=0)-fr_transition.diagonal()
    #transition_toother = fr_transition/othersum_axis0
    #transition_self = fr_transition.diagonal()/(othersum_axis0+fr_transition.diagonal())
    #np.fill_diagonal(transition_toother, 0)